In [1]:
import os

In [2]:
%pwd

'/home/tousside/Documents/10academy/week0/ten-academy-week0/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/tousside/Documents/10academy/week0/ten-academy-week0'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH,
                 ):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)
                create_directories([self.config.artifacts_root])
                

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config
        

In [15]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f"{filename} with following info: \n{headers}")
        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))} bytes")
            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file to the specified directory
        function returns None
        
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-14 10:54:28,931: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-14 10:54:28,932: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-14 10:54:28,933: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-14 10:54:28,934: INFO: common: created directory at: artifacts]
[2025-05-14 10:54:28,934: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-14 10:54:28,935: INFO: 713848397: File already exists with size: ~ 23 KB bytes]
